# <b> Deliverable #2: Data Acquisition </b>
#### Minh Le
#### CS 181/DA 210
#### Dr. Amert
#### April 21th, 2023
---

---

## <b> Part 1: Introduction </b>
The correlation between Gross Domestic Product (GDP) per capita and the life expectancy of citizens has been the subject of interest for various people for many years. While some people may claim that economic growth directly affects human well-being due to the healthcare services provided, it is difficult to conclude whether a higher GDP per capita economy can have an impact on an overall increase in citizen's life expectancy. Therefore, our research aims to resolve this concern:

<b> Central question: </b> Does GDP per capita have a relationship with the life expectancy of countries in the world?

To answer this question, we will use 2 datasets, from Wikipedia and The World Bank website. The average life expectancy at birth in 248 nations throughout the world in 2021 is taken from Wikipedia sources, while the GDP per capita for countries over the world in 2021 is obtained from The World Bank website. These statistics are useful in answering our issue since they provide data on GDP per capita and life expectancy, two crucial factors in analysis. We can determine the relationship between a nation's GDP per capita and its life expectancy by processing the data. Throughout our project, we plan to use graph like scatter plot to observe the correlation.


<b> Reasons for Eligibility: </b>
- Life expectancy dataset: According to the Copyrights section in Terms of Use (https://en.wikipedia.org/wiki/Wikipedia:Copyrights#Guidelines_for_images_and_other_media_files), texts on Wikipedia are co-licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License (CC BY-SA) and the GNU Free Documentation License (GFDL). These licenses allow users to have free access to information stored in Wikipedia, and in turn, they can copy, modify, and redistribute it.

- GDP per capita dataset: According to the Data Access and Licensing page of The World Bank (https://datacatalog.worldbank.org/public-licenses#cc-by), the database provided on this website earns the Creative Commons Attribution 4.0 (CC-BY 4.0) International license. Indeed, the provider enables web users to “copy, modify and distribute data in any format for any purpose, including commercial use.” (The World Bank) To increase the certainty, we also referred to the overall Terms of Use (https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets), which claims to allow users to “extract, download, and make copies of the data contained in the Datasets.” 

- The countries.csv file was downloaded from the Tabular Data portion of the denison.edu website (http://datasystems.denison.edu/data.html). Due to the fact that it was downloaded from a Denison University-owned educational website, this file is permissible for use. We are only allowed to use and alter this file within the confines of our project since we are utilizing it just for educational reasons in a project that a Denison professor assigned to us during the academic year.
---

In [1]:
import os
import io
import sys
import importlib
import pandas as pd
from lxml import etree
import requests
from IPython.display import Image
import os.path
import json
import sqlalchemy as sa

htmlparser =  etree.HTMLParser()

datadir = 'data'

---

## <b> Part 2: Data Acquisition </b>

In this part, we will use web scraping to extract one table of data before converting it to `pandas DataFrame`. For the remaining dataset, we will acquire data and convert it to `pandas DataFrame`.

<b> Note: </b> The XML file downloaded from The World Bank is initially API_NY.GDP.PCAP.CD_DS2_en_xml_v2_5353247.xml. We decided to change the name of the file into `gdp.xml` for convenience.

### <b> Life Expectancy Dataset: </b>

The information on this dataset:
- Size: 248 rows and 5 columns. 
- Column names: `Countries and Region` (char), `All` (float), `M` (float), `F` (float), and `Gender life gap` (float)
- Entity: Life expectancy
- Format: HTML file

We will web scraping to extract life expectancy dataset using the `curl` command. The following command will save the HTML source of https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy?fbclid=IwAR29IP7YamZdMFABYdreOxhilZWWQzS7KoDxQKCbgTv3Hl5PW7jS_IH5r_g#CIA_World_Factbook_(2014-2020) to my local computer, in a file `life.html` in the same folder as this notebook.

In [2]:
#Download the HTML to a file life.html
#!curl -s -o life.html https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy?fbclid=IwAR29IP7YamZdMFABYdreOxhilZWWQzS7KoDxQKCbgTv3Hl5PW7jS_IH5r_g#CIA_World_Factbook_(2014-2020)

After we have extract data to our local computer, we will now parse the data using `xpath` and convert them to a `pandas DataFrame`. First of all, we will find the right table node (2021 table) and assign it to the variable `tableroot`.

In [3]:
#parse data set
path = os.path.join(datadir, 'life.html')
htmlparser = etree.XMLParser(remove_blank_text=True)
life = etree.parse(path, htmlparser)

#get the root of the file
life_root = life.getroot()

#get the root of the table 
tableroot = life_root.xpath("//table/caption[text() = 'Countries and regions by life expectancy at birth in 2021 (2022 report)']/..")[0]

Extract the column names from the table to the variable `col_names` using `xpath`.

<b> Note: </b> When using `xpath`, we will encounter irrelevant values, so we will use list comprehension to cope with this problem

In [4]:
#extract the column name from the table
col_names = tableroot.xpath(".//th//text()")

#using list comprehension to deal with irrelevant values
col_names = [col_names[i] for i in range(len(col_names[:6])) if i != 1]

#display column name
col_names

['Countries and regions', 'All', 'M', 'F', 'Gender life gap']

When collecting data from the HTML file, we find some bizzare elements, such as `th` and `td` being children of a specific `tr` (These rows contain category names like Asia or High Income countries, which do not represent any country). Therefore, we will use procedural operations to add each row of the dataset in to a LoL.

In [5]:
#initialize LoL
LoL = []

for child in tableroot.find("tbody").findall("tr"):
    if child.find("th") is not None: #ignore the row that has both th and tr
        continue
    else:
        #country row will have a child 'a' of td, while the other rows do not have
        a = [val.text if val.find("a") is None else val.find("a").text for val in child.iter("td")]
    LoL.append(a)

Now, we will add the LoL and the column names into `pandas DataFrame`:

In [6]:
life_df = pd.DataFrame(LoL, columns=col_names)
life_df.head(10)

,Countries and regions,All,M,F,Gender life gap
0,Afghanistan,62.0,58.9,65.3,6.4
1,Albania,76.5,74.1,79.2,5.1
2,Algeria,76.4,74.9,78.0,3.1
3,American Samoa,72.5,69.1,76.6,7.5
4,Andorra,80.4,77.2,84.3,7.1
5,Angola,61.6,59.0,64.3,5.3
6,Anguilla,76.6,73.2,80.5,7.3
7,Antigua and Barbuda,78.5,75.8,80.9,5.1
8,Argentina,75.4,72.2,78.6,6.4
9,Armenia,72.0,66.6,77.4,10.8


Here are the first 10 rows of the table. For Deliverable #3, we intend to use these columns: 
- `Countries and regions`: Countries (string)
- `F`: The life expectancy at birth for female (float)
- `M`: The life expectancy at birth for male (float)

### <b> GDP Per Capita Dataset: </b>

Regarding this dataset, we will acquire the data on GDP per capita in 2021 from The World Bank source by using XML parser and procedural operations. We will then convert it to `pandas DataFrame`.

The information on this dataset:
- Size: 278 rows and 3 columns
- Column names: `Country` (char), `Most Recent Year` (int), and `Most Recent Value` (float)
- Entity: GDP per capita
- Format: XML file 


In [7]:
# Use XML parser to parse data
path = os.path.join(datadir, 'gdp.xml')
myparser = etree.XMLParser(remove_blank_text=True)
tree = etree.parse(path, myparser)

# Obtain the root
gdp_root = tree.getroot()

<b> Note: </b> We notice that there are 4 fields that correspond to 4 attribute values for each country or region in the XML file. However, there are only 3 columns included in the version that is shown on the website. The attribute in the XML file with the value `Item` represents the hidden column. This tag's wording reads `GDP per capita (current US$)`. 

First of all, we will extract the column names using `xpath`.

In [8]:
#extract the column name from the table
col_names_gdp = gdp_root.xpath("//record[position() = 1]/field/@name")

#display column name
col_names_gdp

['Country or Area', 'Item', 'Year', 'Value']

We will now select add all the values into the DoL.

In [9]:
#initialize a DoL
DoL = {key:[] for key in col_names_gdp}


# Iterate through all the records in the XML
for record in gdp_root.findall(".//record"):
    # Extract the values of each field
    country = record.find("./field[@name='Country or Area']").text
    item = record.find("./field[@name='Item']").text
    year = record.find("./field[@name='Year']").text
    value = record.find("./field[@name='Value']").text
    
    # Append the values to their respective lists
    DoL["Country or Area"].append(country)
    DoL["Item"].append(item)
    DoL["Year"].append(year)
    DoL["Value"].append(value)

# Create a pandas DataFrame from the lists
gdp_per_capita_df = pd.DataFrame(DoL)

#filter out the table so that the table only have 2021 data
gdp_per_capita_df = gdp_per_capita_df[gdp_per_capita_df.Year == '2021'].copy()

#display the DataFrame
gdp_per_capita_df.head(10)

,Country or Area,Item,Year,Value
61,Aruba,GDP per capita (current US$),2021,29342.1008575886
123,Africa Eastern and Southern,GDP per capita (current US$),2021,1549.77272965144
185,Afghanistan,GDP per capita (current US$),2021,368.754614175459
247,Africa Western and Central,GDP per capita (current US$),2021,1757.03062622138
309,Angola,GDP per capita (current US$),2021,1953.53375721508
371,Albania,GDP per capita (current US$),2021,6492.87201224634
433,Andorra,GDP per capita (current US$),2021,42137.3272710372
495,Arab World,GDP per capita (current US$),2021,6271.31900080308
557,United Arab Emirates,GDP per capita (current US$),2021,44315.5541834116
619,Argentina,GDP per capita (current US$),2021,10636.1201956183


Here are the first 10 rows of the table. For Deliverable #3, we intend to use these columns:
- `Country or Area`: Countries (string)
- `Value`: The value of GDP per capita (float)

### <b> Countries Dataset: </b>

Regarding this dataset, we will acquire the data from the `countries.csv` file that we have used in class (http://datasystems.denison.edu/data.html). We will read the data using `read_csv` and convert it to `pandas DataFrame`.

The information on this dataset:
- Size: 217 rows and 5 columns
- Column names: `code` (char), `name` (char), `currency` (char), `region` (char), `income` (char)
- Entity: countries
- Format: CSV file 

In [10]:
#read in the file and assign it to countries_df
filepath = os.path.join(datadir, 'countries.csv')
countries_df = pd.read_csv(filepath)
countries_df.head(10)

,code,name,currency,region,income
0,ABW,Aruba,Aruban florin,Latin America & Caribbean,High income
1,AFG,Afghanistan,Afghan afghani,South Asia,Low income
2,AGO,Angola,Angolan kwanza,Sub-Saharan Africa,Lower middle income
3,ALB,Albania,Albanian lek,Europe & Central Asia,Upper middle income
4,AND,Andorra,Euro,Europe & Central Asia,High income
5,ARE,United Arab Emirates,U.A.E. dirham,Middle East & North Africa,High income
6,ARG,Argentina,Argentine peso,Latin America & Caribbean,Upper middle income
7,ARM,Armenia,Armenian dram,Europe & Central Asia,Upper middle income
8,ASM,American Samoa,U.S. dollar,East Asia & Pacific,Upper middle income
9,ATG,Antigua and Barbuda,East Caribbean dollar,Latin America & Caribbean,High income


---
## <b> Part 4: Tidying Data </b>

In this part, we will tidy the dataset. Here are some of the features that make the data untidy:
- The name for some countries are not the same in three files `gdp_xml`, `countries_csv`, and `life.html` (The Bahamas vs. Bahamas, The), so we will manually change the name to reach a consistency between tables.
- The `Item` and `Value` columns in `gdp_per_capita_df` are both generic column names.
- As we only use year 2021, we do not need to keep the `Year` column in `life_df`.
- In `life_df`, we have found that `Gender life gap` and `All` columns can be derived from `M` and `F` columns. Therefore, we will only keep the `All` column and change the name of this column.
- We will change the column named `Country and regions` of `life_df` into `country_and_area`.
- We will change the column named `Country or Area` of `gdp_per_capita_df` into `country_and_area`.
- In `gdp_per_capita`, some of the rows in the `Country or Area` column are not exactly countries (ex: `World, Upper Middle Income, etc`). Therefore, when combining two tables we will use `right join` (only take the rows from `life_df` since all the rows from this dataset represent country names) to make it tidy.
- We will not use the columns `code`, `income` and `currency` in the `countries_df` so we will drop these columns and we will change the column named `name` into `country`.
- Since the two tables have the same set of independent variables (`country_and_area`), we will combine two tables into a single one.

In [11]:
#drop columns
gdp_per_capita_df = gdp_per_capita_df.drop(['Item', 'Year'], axis=1)

#rename columns
gdp_per_capita_df.rename(columns={'Country or Area': 'country_and_area', 'Value': 'gdp_per_capita'}, inplace=True)

#set index
gdp_per_capita_df.set_index('country_and_area', inplace=True)

In [12]:
#drop columns
life_df = life_df.drop(['F', 'M', 'Gender life gap'], axis=1)

#rename columns
life_df.rename(columns={'Countries and regions': 'country_and_area', 'All': 'life'}, inplace=True)

#set index
life_df.set_index('country_and_area', inplace=True)

In [13]:
#drop columns
countries_df = countries_df.drop(['code', 'currency', 'income'], axis=1)

#rename columns
countries_df.rename(columns={'name': 'country_and_area'}, inplace=True)

#set index
countries_df.set_index('country_and_area', inplace=True)

In [14]:
#join two tables
life_gdp = gdp_per_capita_df.join(life_df, how='right')

#display the dataset
life_gdp

,gdp_per_capita,life
country_and_area,,
Afghanistan,368.754614175459,62.0
Albania,6492.87201224634,76.5
Algeria,3690.62787797599,76.4
American Samoa,15743.3107582991,72.5
Andorra,42137.3272710372,80.4
...,...,...
Wallis and Futuna,NaN,78.4
Western Sahara,NaN,70.8
Yemen,NaN,63.8


<b> Note: </b> We found that the `life_gdp` table and the `countries_df` table have the same independent variable, which is the country and area name. Therefore, we will join two tables together with a `left join` that only takes the rows from `countries_df`. The reason why we do this is that some of the rows in `life_gdp` table from the `country_and_area` column are island territories (ex: `Angulia, or Mayotte`), which will not appear in `countries_df`. We decide to only take the rows from `countries_df` so that we can remove some outliers for our analysis (since only a few people live on the island so it is hard to find the relationship between gdp and life expectancy based on these data)

In [15]:
#join two tables
indicators = countries_df.join(life_gdp, how='left')

#reset index
indicators.reset_index(inplace=True)

#display the dataset
indicators

,country_and_area,region,gdp_per_capita,life
0,Aruba,Latin America & Caribbean,29342.1008575886,74.6
1,Afghanistan,South Asia,368.754614175459,62.0
2,Angola,Sub-Saharan Africa,1953.53375721508,61.6
3,Albania,Europe & Central Asia,6492.87201224634,76.5
4,Andorra,Europe & Central Asia,42137.3272710372,80.4
...,...,...,...,...
212,Kosovo,Europe & Central Asia,5269.78390114389,76.8
213,Yemen,Middle East & North Africa,NaN,63.8
214,South Africa,Sub-Saharan Africa,7055.04477598783,62.3
215,Zambia,Sub-Saharan Africa,1137.34363269164,61.2


**Table-field Design:**

We made one table `indicators` in the database:
- `indicators`: In this table, the primary key are `country_and_area`.

**Sound Database Design**
1. Tables names: `indicators` is a plural name. 
2. Primary keys: `indicators` has a singleton key, which is `country_and_area`
3. Functional dependency: `country_and_area -> region, gdp_per_capita, life`. 

**Reasons for Adherence to Principles of Good Database Design:**

- No values are set up as field.
- No mashups.
- No redundant information.
- No derived field.

Set Credentials

In [16]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [17]:
def buildConnectionString(source="sqlite_country"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

In [18]:
# Build the conection string
cstring = buildConnectionString("sqlite_country")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./country.db


We will now create table `indicators` on SQLite.

In [19]:
query = """
CREATE TABLE IF NOT EXISTS indicators (
    country_and_area VARCHAR(64) NOT NULL PRIMARY KEY,
    region VARCHAR(64) NULL,
    gdp_per_capita FLOAT NULL,
    life FLOAT NULL
)
"""
#Execute
try:
    connection.execute(query)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of indicators failed:", str(err))

Insert data to the table.

In [20]:
indicators.to_sql("indicators", con=connection, if_exists="append", index=False)

217

Create a DataFrame named `indicators_df` from the table

In [21]:
indicators_df = pd.read_sql_table('indicators', connection)
indicators_df.head()

,country_and_area,region,gdp_per_capita,life
0,Aruba,Latin America & Caribbean,29342.100858,74.6
1,Afghanistan,South Asia,368.754614,62.0
2,Angola,Sub-Saharan Africa,1953.533757,61.6
3,Albania,Europe & Central Asia,6492.872012,76.5
4,Andorra,Europe & Central Asia,42137.327271,80.4


In [22]:
# Close the connection!
try:
    connection.close()
except:
    pass
del engine

---
---

## <b> Reference: </b>

- GDP Per Capita dataset: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=2019&start=1960
- Life Expectancy dataset: https://en.wikipedia.org/wiki/List_of_countries_by_life_expectancy#World_Health_Organization_(2019)
- Countries dataset: http://datasystems.denison.edu/data.html
